In [ ]:
#Imports
#install.packages("glmnet")
library(mgcv)
library(glmnet)

data <- read.csv("/content/red_wine.csv")
n = length(data[, "quality"])
#Split data into test and training
train <- data[1:1000, c("quality", "alcohol", "sulphates", "pH", "density", "total.sulfur.dioxide", "free.sulfur.dioxide", "chlorides", "residual.sugar", "citric.acid", "volatile.acidity", "fixed.acidity")]
test <- data[1001:n, c("quality", "alcohol", "sulphates", "pH", "density", "total.sulfur.dioxide", "free.sulfur.dioxide", "chlorides", "residual.sugar", "citric.acid", "volatile.acidity", "fixed.acidity")]



#Dependent var
train_quality <- train$quality
test_quality <- test$quality
train <- subset(train, select = -quality)
test <- subset(test, select = -quality)

#Standardize the variables
train<- as.data.frame(scale(train))
test<- as.data.frame(scale(test))

train$quality <- train_quality
test$quality <- test_quality


# Parametric logistic on all vars

In [ ]:
#Estimate parametric logistic with ML, all parameters, logit link

est_par <- gam(quality~alcohol+sulphates+pH+density+total.sulfur.dioxide+
  free.sulfur.dioxide+chlorides+residual.sugar+citric.acid+volatile.acidity+fixed.acidity,family = binomial, data = data.frame(train))
summary(est_par)


Family: binomial 
Link function: logit 

Formula:
quality ~ alcohol + sulphates + pH + density + total.sulfur.dioxide + 
    free.sulfur.dioxide + chlorides + residual.sugar + citric.acid + 
    volatile.acidity + fixed.acidity

Parametric coefficients:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)           0.006863   0.078475   0.087  0.93031    
alcohol               0.869931   0.130246   6.679 2.40e-11 ***
sulphates             0.446117   0.090677   4.920 8.66e-07 ***
pH                    0.038712   0.140991   0.275  0.78365    
density              -0.094529   0.180091  -0.525  0.59965    
total.sulfur.dioxide -0.790634   0.135026  -5.855 4.76e-09 ***
free.sulfur.dioxide   0.301224   0.116204   2.592  0.00954 ** 
chlorides            -0.127203   0.091291  -1.393  0.16351    
residual.sugar        0.066750   0.095963   0.696  0.48669    
citric.acid          -0.295866   0.142297  -2.079  0.03760 *  
volatile.acidity     -0.579001   0.108628  -5.330 9.

In [ ]:
#Remove y from df=> train == X
train <- subset(train, select = -quality)

#Ridge
#Select Lambda with CV, leave-one-out method
cv.log.ridge <- cv.glmnet(train, train_quality, family="binomial", alpha = 0, nfolds=1000, grouped=FALSE)
print(cv.log.ridge)
#Ridge regression with logistic link
log.ridge <- glmnet(train, train_quality, family="binomial", alpha = 0,lambda = cv.log.ridge$lambda.min)
coef(log.ridge)



Call:  cv.glmnet(x = train, y = train_quality, nfolds = 1000, grouped = FALSE,      family = "binomial", alpha = 0) 

Measure: Binomial Deviance 

     Lambda Index Measure      SE Nonzero
min 0.02069   100   1.066 0.02848      11
1se 0.09167    84   1.094 0.02201      11


12 x 1 sparse Matrix of class "dgCMatrix"
                               s0
(Intercept)           0.008360577
alcohol               0.733896285
sulphates             0.382170990
pH                    0.044049101
density              -0.107976636
total.sulfur.dioxide -0.610809086
free.sulfur.dioxide   0.173415295
chlorides            -0.132109795
residual.sugar        0.062492271
citric.acid          -0.134095450
volatile.acidity     -0.461609207
fixed.acidity         0.242328645

In [ ]:
#Select Lambda with CV, leave-one-out method
cv.log.lasso <- cv.glmnet(train, train_quality, family="binomial", alpha = 1, nfolds=1000, grouped=FALSE)
print(cv.log.lasso)
#Lasso regression with logistic link
log.lasso <- glmnet(train, train_quality, family="binomial", alpha = 1,lambda = cv.log.lasso$lambda.min)
coef(log.lasso)


Call:  cv.glmnet(x = train, y = train_quality, nfolds = 1000, grouped = FALSE,      family = "binomial", alpha = 1) 

Measure: Binomial Deviance 

     Lambda Index Measure      SE Nonzero
min 0.00261    48   1.062 0.03151       9
1se 0.03219    21   1.091 0.02304       4


12 x 1 sparse Matrix of class "dgCMatrix"
                               s0
(Intercept)           0.006386795
alcohol               0.887803823
sulphates             0.407921137
pH                    .          
density               .          
total.sulfur.dioxide -0.745106437
free.sulfur.dioxide   0.259189220
chlorides            -0.128451740
residual.sugar        0.022722894
citric.acid          -0.208655267
volatile.acidity     -0.541028141
fixed.acidity         0.174079881

In [ ]:
#Predict for the ML model
y_pred_par <- predict(est_par, newdata = test)
#Remove y from df => test == ~X
test <- subset(test, select = -quality)
#Assign test into a matrix
mxval <- data.matrix(test)
#Predict for the lasso model
y_pred_lasso <- predict(log.lasso, mxval)
#exp(y_pred_lasso)/(1+exp(y_pred_lasso))

#Predict for the ridge model
y_pred_ridge <- predict(log.ridge, mxval)
#exp(y_pred_ridge)/(1+exp(y_pred_ridge))

In [ ]:
# Perform a binary assignment, <- 1 if the logistic transformation <=0, 0 else for all data points
bin_ridge <- c(rep(0, n-1000))
for(i in c(1:599)){

  if (exp(y_pred_ridge[i])/(1+exp(y_pred_ridge[i])) <= 0.5){
      bin_ridge[i] = 0
    }
    else{
      bin_ridge[i] = 1
    }
  }


bin_lasso <- c(rep(0, n-1000))
for(i in c(1:599)){
  if (exp(y_pred_lasso[i])/(1+exp(y_pred_lasso[i])) <= 0.5){
     bin_lasso[i] = 0
    }
    else{
      bin_lasso[i] = 1
    }
  }


bin_par <- c(rep(-1000, n-1000))
for(i in c(1:599)){
  if (exp(y_pred_par[i])/(1+exp(y_pred_par[i])) <= 0.5){
     bin_par[i] = 0
    }
    else{
      bin_par[i] = 1
    }
  }
#Perform MSE
MSE <-function(y, y_pred){
  1/length(y_pred) * sum((y-y_pred)^2)
}

MSE(test_quality, bin_lasso)
MSE(test_quality, bin_ridge)
MSE(test_quality, bin_par)

#Percentage correctly predicted, since y and predicted y are strictly binary, 1 - MSE == probability of correct classification
1-MSE(test_quality, bin_lasso)
1- MSE(test_quality, bin_ridge)
1- MSE(test_quality, bin_par)



[1] 0.2604341

[1] 0.2570952

[1] 0.2470785

[1] 0.7395659

[1] 0.7429048

[1] 0.7529215